In [5]:
# !pip install python-dotenv==0.21.0
# !pip show python-dotenv
# !pip install undetected-chromedriver==3.5.5
# !pip show undetected-chromedriver

# Extract more information from linkedin ( linkedin url, profiles, size, ...)

In [1]:
from Item import Item, Job, Company
from ItemStorage import ItemStorage
from LinkedinAutomation import LinkedinAutomation
import time, random
import os
from dotenv import load_dotenv
import copy

In [3]:
# Charger les variables d'environnement depuis le fichier .env
load_dotenv('settings.env')

# Accéder aux variables d'environnement
email = os.getenv('LINKEDIN_EMAIL')
pwd = os.getenv('LINKEDIN_PWD')

linkedinAutomation = LinkedinAutomation()
linkedinAutomation.driver.quit()

In [5]:
import undetected_chromedriver as uc
driver = uc.Chrome()
linkedinAutomation.driver = driver

In [7]:
linkedinAutomation.get_linkedin_authentication(email = email, pwd = pwd)

linkedin_authentication --> yes


In [13]:
#linkedinAutomation.send_connection_invitation_from_profile_url('https://www.linkedin.com/in/taoufyq-elansari-a59626111/')

In [19]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

def extract_job_offers_from_key_words(self, key_words='commercial', country='France'):
    try:
        if self.driver.current_url != "https://www.linkedin.com/jobs/":
            self.driver.get("https://www.linkedin.com/jobs/")
        input_key_words = self.get_element('//div[contains(@class, "jobs-search-box__input jobs-search-box__input--keyword jobs-search-box__input--both-bars jobs-search-box__input--clear-text")]/div/div/input')
        if input_key_words["status"]:
            input_key_words = input_key_words["data"]
            input_key_words.clear()
            time.sleep(3)
            input_key_words.send_keys(key_words)
            #print(input_key_words.text)
        else:
            print({"status": False, "data":input_key_words["data"] })
        time.sleep(3)
        
        input_country = self.get_element('//div[contains(@class, "jobs-search-box__input jobs-search-box__input--location jobs-search-box__input--both-bars jobs-search-box__input--clear-text")]/div/div/input')
        if input_country["status"]:
            input_country = input_country["data"]
            input_country.send_keys(Keys.NULL)
            input_country.clear()
            time.sleep(3)
            input_country.send_keys(country)
            time.sleep(3)
            #print(input_country.text)
            input_country.send_keys(Keys.ENTER)
        else:
            print({"status": False, "data":input_country["data"] })
            
        return {"status": True, "data": 'data' }
    except Exception as e:
        print(f'ERROOOR at extract_job_offers_from_key_words: {e}')
        return {"status": False, "data": str(e) }
        
key_words='Directeur Commercial'
country='France'
extract_job_offers_from_key_words(linkedinAutomation, key_words=key_words, country=country)

{'status': True, 'data': 'data'}

In [21]:
self = linkedinAutomation

def extract_job_details(self):
        job_details = {}
        def remove_after_string(a, b):
            index = a.find(b)
            if index != -1:
                return a[:index + len(b)-len(b)]
            return a
        div_details = self.get_element('//div[contains(@class, "jobs-search__job-details--wrapper")]')
        if div_details["status"]:
            div_details = div_details["data"]
        else:
            print({"status": False, "data": 'Unable to locate element: div_details' })

        # print(f"*"*150)
        a_company = self.get_element('div[2]/div/div[1]/div/div[1]/div/div[1]/div/div[1]/div[1]/div/a', from_elem=div_details)
        if not a_company["status"]:
            a_company = self.get_element('div[2]/div/div[1]/div/div[1]/div/div[1]/div/div[1]/div[1]/div', from_elem=div_details)
        if a_company["status"]:
            a_company = a_company["data"]
            job_details['company_name'] = a_company.get_attribute('innerText')
            job_details['company_linkedin_url'] = a_company.get_attribute('href')
            # print(company_name)
            # print(company_linkedin_url)
        else:
            print({"status": False, "data": 'Unable to locate element: a_company' })

        # print(f"*"*150)
        job_inf = self.get_element('div[2]/div/div[1]/div/div[1]/div/div[1]/div/div[3]/div', from_elem=div_details)
        if job_inf["status"]:
            job_inf = job_inf["data"]
            job_details['job_inf'] = job_inf.get_attribute('innerText')
            # print(job_inf)
        else:
            print({"status": False, "data": 'Unable to locate element: job_inf' })

        # print(f"*"*150)
        job_title = self.get_element('div[2]/div/div[1]/div/div[1]/div/div[1]/div/div[2]/div/h1/a', from_elem=div_details)
        if job_title["status"]:
            job_title = job_title["data"]
            job_details['job_url'] = job_title.get_attribute('href')
            job_details['job_title'] = job_title.get_attribute('innerText')
            # print(job_title)
            # print(job_url)
        else:
            print({"status": False, "data": 'Unable to locate element: job_title' })

        # print(f"*"*150)
        job_mode = self.get_element('div[2]/div/div[1]/div/div[1]/div/div[1]/div/div[4]/ul/li[1]', from_elem=div_details)
        if job_mode["status"]:
            job_mode = job_mode["data"]
            job_mode = job_mode.get_attribute('innerText')
            job_details['job_mode'] = ",, ".join([e.strip() for e in job_mode.split('\n') if "Correspond à vos préférences d’emploi," not in e and e.strip()])
            # print(job_mode)
        else:
            print({"status": False, "data": 'Unable to locate element: job_mode' })

        # print(f"*"*150)
        company_size = self.get_element('div[2]/div/div[1]/div/section/section/div[1]/div[2]/span', from_elem=div_details)
        if company_size["status"]:
            company_size = company_size["data"]
            job_details['company_size'] = company_size.text
            #print(company_size)
        else:
            print({"status": False, "data": 'Unable to locate element: company_size' })
    
        # print(f"*"*150)
        Secteur = self.get_element('div[2]/div/div[1]/div/section/section/div[1]/div[2]', from_elem=div_details)
        if Secteur["status"]:
            Secteur = Secteur["data"]
            Secteur = Secteur.text
            job_details['Secteur'] = remove_after_string(Secteur, job_details['company_size'])
            # print(Secteur)
        else:
            print({"status": False, "data": 'Unable to locate element: Secteur' })

        # print(f"*"*150)
        job_skills = None
        job_skills_1 = self.get_element('//div[1][contains(@class, "job-details-how-you-match__skills-item-wrapper display-flex flex-row")]/div/a', from_elem=div_details)
        if job_skills_1["status"]:
            job_skills_1 = job_skills_1["data"]
            job_skills_1 = job_skills_1.text
            job_skills = job_skills_1
        else:
            print({"status": False, "data": 'Unable to locate element: job_skills_1' })
                                
        job_skills_2 = self.get_element('//div[2][contains(@class, "job-details-how-you-match__skills-item-wrapper display-flex flex-row")]/div/a', from_elem=div_details)
        if job_skills_2["status"]:
            job_skills_2 = job_skills_2["data"]
            job_skills_2 = job_skills_2.text
            job_skills = f'{job_skills},,, {job_skills_2}' if job_skills else job_skills_2
        else:
            print({"status": False, "data": 'Unable to locate element: job_skills_2' })

        job_details['job_skills'] = job_skills
        
        # print(f"*"*150)
        job_description = self.get_element('//article/div/div[1]', from_elem=div_details)
        if job_description["status"]:
            job_description = job_description["data"]
            job_details['job_description'] = job_description.get_attribute('innerText')
            # print(job_description)
        else:
            print({"status": False, "data": 'Unable to locate element: job_description' })

        return job_details

job_details = extract_job_details(self)
job_details

{'status': False, 'data': 'Unable to locate element: job_skills_1'}
{'status': False, 'data': 'Unable to locate element: job_skills_2'}


{'company_name': 'Expectra',
 'company_linkedin_url': 'https://www.linkedin.com/company/expectra/life',
 'job_inf': 'Puteaux, Île-de-France, France · il y a 22 heures · 23\xa0candidats',
 'job_url': 'https://www.linkedin.com/jobs/view/4102278487/?alternateChannel=search&refId=XCBi%2B%2FWzqUCrtYwVJJSigw%3D%3D&trackingId=5h7SN0xgdzK62UMijolZPg%3D%3D&trk=d_flagship3_search_srp_jobs',
 'job_title': 'Senior Sales Executive',
 'job_mode': 'Hybride  Temps plein  Cadre',
 'company_size': '201-500 employés',
 'Secteur': 'Services de ressources humaines ',
 'job_skills': None,
 'job_description': "À propos de l’offre d’emploi\n\nNotre client est un éditeur européen de plateforme digitale de gestion intégrée des risques les plus complètes du marché. Ses interlocuteurs sont les Directeurs Juridiques, Directeurs de la Conformité, DSI, RSSI, Directeurs de l’Audit ou Directeurs financiers.\n\nEn forte croissance avec de belles opportunités d’évolutions, en 4 ans d’existence, la société a conquis 16% 

In [23]:
def extract_list_of_job_offers(self, key_words):
    try:
        list_offers = self.get_element('//li[contains(@class, "scaffold-layout__list-item")]', group=True)
        if list_offers["status"]:
            list_offers = list_offers["data"]
            print(len(list_offers))
            job_items = []
            for list_offer in list_offers:
                try:
                    list_offer.click()
                    time.sleep(random.uniform(3,5))
                    # job_details = self.extract_job_details()
                    job_details = extract_job_details(linkedinAutomation)
                    job_item = Item(job_details)
                    job_item.key_words = key_words
                    job_items.append(job_item)
                    time.sleep(random.uniform(0.5,1.5))
                except Exception as e:
                    print(e)
            ItemStorage(file_path='./results/job_offers_2_18-12-2024.xlsx', value=job_items)
        else:
            print({"status": False, "data":list_offers["data"] })

        return {"status": True, "data": job_items }
    except Exception as e:
        print(f'ERROOOR at extract_list_of_job_offers: {e}')
        return {"status": False, "data": str(e) }

job_items_result = extract_list_of_job_offers(linkedinAutomation, key_words)

25
{'status': False, 'data': 'Unable to locate element: job_skills_1'}
{'status': False, 'data': 'Unable to locate element: job_skills_2'}
{'status': False, 'data': 'Unable to locate element: job_skills_2'}
{'status': False, 'data': 'Unable to locate element: job_skills_1'}
{'status': False, 'data': 'Unable to locate element: job_skills_2'}
{'status': False, 'data': 'Unable to locate element: job_skills_1'}
{'status': False, 'data': 'Unable to locate element: job_skills_2'}
{'status': False, 'data': 'Unable to locate element: job_skills_1'}
{'status': False, 'data': 'Unable to locate element: job_skills_2'}
{'status': False, 'data': 'Unable to locate element: job_skills_1'}
{'status': False, 'data': 'Unable to locate element: job_skills_2'}
{'status': False, 'data': 'Unable to locate element: job_skills_1'}
{'status': False, 'data': 'Unable to locate element: job_skills_2'}
{'status': False, 'data': 'Unable to locate element: job_skills_1'}
{'status': False, 'data': 'Unable to locate e

In [ ]:
# ItemStorage(file_path='./results/job_offers_2_18-12-2024.xlsx', value=job_items_result['data'])

In [23]:
next_button = self.get_element('//button[@aria-label="Voir la page suivante" and contains(@class, "jobs-search-pagination__button--next")]')
if next_button["status"]:
    next_button = next_button["data"]
    next_button.click()
else:
    print(next_button)

{'status': False, 'data': 'Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@aria-label="Voir la page suivante" and contains(@class, "jobs-search-pagination__button--next")]"}\n  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception\nStacktrace:\n\tGetHandleVerifier [0x00E134A3+25059]\n\t(No symbol) [0x00D9CEA4]\n\t(No symbol) [0x00C7BEC3]\n\t(No symbol) [0x00CBFD86]\n\t(No symbol) [0x00CBFFCB]\n\t(No symbol) [0x00CFD952]\n\t(No symbol) [0x00CE1F44]\n\t(No symbol) [0x00CFB51E]\n\t(No symbol) [0x00CE1C96]\n\t(No symbol) [0x00CB3FAC]\n\t(No symbol) [0x00CB4F3D]\n\tGetHandleVerifier [0x01105613+3113811]\n\tGetHandleVerifier [0x0111A2DA+3199002]\n\tGetHandleVerifier [0x01112AB2+3168242]\n\tGetHandleVerifier [0x00EB3310+680016]\n\t(No symbol) [0x00DA57ED]\n\t(No symbol) [0x00DA2A98]\n\t(No symbol) [0x00DA2C35]\n\t(No symb

In [29]:
job_offers.keys()

dict_keys(['company_name', 'company_linkedin_url', 'job_inf', 'job_url', 'job_title', 'job_mode', 'company_size', 'Secteur', 'job_skills', 'job_description'])

In [5]:
import pandas as pd

df = pd.read_excel('./results/job_offers_2_18-12-2024_13h00min.xlsx')
unique_jobs = df.drop_duplicates(subset=['company_name','job_title'])
unique_jobs

,company_name,company_linkedin_url,job_inf,job_url,job_title,job_mode,company_size,Secteur,job_skills,job_description,key_words
0,CRISTAL DECISIONS,https://www.linkedin.com/company/cristal-decis...,"Boulogne-Billancourt, Île-de-France, France · ...",https://www.linkedin.com/jobs/view/4086187766/...,Business developer spécialisé Mobilité,Sur site Temps plein Confirmé,11-50 employés,Services et conseil aux entreprises,NaN,À propos de l’offre d’emploi\n\n✨ Qui sommes-n...,business developer freelance
1,La Relève,https://www.linkedin.com/company/la-releve/life,Paris et périphérie · il y a 1 semaine · 35 au...,https://www.linkedin.com/jobs/view/4092313695/...,CDI - Business Developer Enterprise (LegalTech...,Hybride Temps plein Premier emploi,11-50 employés,Recrutement et placement de personnel,"Gestion de la relation client (CRM) et Vente,,...",À propos de l’offre d’emploi\n\nNous recrutons...,business developer freelance
2,Black Belt,https://www.linkedin.com/company/black-belt-io...,"Île-de-France, France · il y a 5 jours · 36 ca...",https://www.linkedin.com/jobs/view/4098199516/...,Business Developer freelance,Sur site Temps plein Confirmé,11-50 employés,Recrutement et placement de personnel,"Business, Développement commercial, Gestion de...",À propos de l’offre d’emploi\n\nOpportunité de...,business developer freelance
3,Elwing,https://www.linkedin.com/company/elwing/life,"Bordeaux, Nouvelle-Aquitaine, France · il y a ...",https://www.linkedin.com/jobs/view/4099765254/...,Business Development B2B,Hybride Temps plein,2-10 employés,Fabrication d’articles de sport,"Développement commercial, Gestion de la relati...",À propos de l’offre d’emploi\n\nObjectifs\nPar...,business developer freelance
4,UPTOO,https://www.linkedin.com/company/uptoo/life,"Créteil, Île-de-France, France · il y a 16 heu...",https://www.linkedin.com/jobs/view/4102588635/...,Business developer - Marketplace Immobilier (H/F),Hybride Temps plein Confirmé,201-500 employés,Services et conseil aux entreprises,"Développement commercial, Gestion de la relati...",À propos de l’offre d’emploi\n\nÊtes-vous prêt...,business developer freelance
...,...,...,...,...,...,...,...,...,...,...,...
157,Bras Droit des Dirigeants,https://www.linkedin.com/company/bras-droit-de...,"Argenteuil, Île-de-France, France · il y a 11 ...",https://www.linkedin.com/jobs/view/4104602535/...,DIRECTEUR COMMERCIAL INDEPENDANT H/F H/F,Sur site Cadre,51-200 employés,Services et conseil aux entreprises,NaN,À propos de l’offre d’emploi\n\nMISSIONS Rejoi...,Directeur Commercial
160,METEOJOB by CleverConnect,https://www.linkedin.com/company/meteojob-by-c...,"Saint-Marcel-lès-Valence, Auvergne-Rhône-Alpes...",https://www.linkedin.com/jobs/view/4104720548/...,Directeur d'agence - commercial H/F - Saint-Ma...,Sur site Directeur,51-200 employés,Recrutement et placement de personnel,NaN,À propos de l’offre d’emploi\nCette offre d’em...,Directeur Commercial
161,E.Leclerc Bordeaux-Chartrons,https://www.linkedin.com/company/e-leclerc-bor...,"Linxe, Nouvelle-Aquitaine, France · il y a 7 h...",https://www.linkedin.com/jobs/view/4103624588/...,DIRECTRICE/DIRECTEUR COMMERCIAL(E) SUPERMARCHE...,Sur site Directeur,201-500 employés,Commerce de gros de produits alimentaires et b...,"Conseil aux clients, Diffusion continue en dir...",À propos de l’offre d’emploi\n\nDescription\n\...,Directeur Commercial
162,Veolia Ecuador,https://www.linkedin.com/company/veolia-ecuado...,"Vénissieux, Auvergne-Rhône-Alpes, France · il ...",https://www.linkedin.com/jobs/view/4103408322/...,Directeur Commercial H/F,Sur site Directeur,1 001-5 000 employés,Services de conseil en environnement,NaN,À propos de l’offre d’emploi\n\nDescription de...,Directeur Commercial


In [45]:
def is_for_freelancer(job_description, freelancer_keywords=None):  
    if freelancer_keywords is None:
        freelancer_keywords = [
            "freelance", "temporary contract", "independent contractor",
            "contrat temporaire", "indépendant", "indépendante", "independant", "independante", "entrepreneur"
        ]
    # Convert everything to lowercase for case-insensitive comparison
    job_description = job_description.lower()
    # Check for the presence of keywords
    for keyword in freelancer_keywords:
        if keyword.lower() in job_description:
            return 'freelance'
    return None
    
def extract_job_contract_type(ligne):
    job_description = f'{ligne['job_title']} {ligne['job_mode']}'
    result = None
    for_freelancer = is_for_freelancer(job_description)
    result = for_freelancer if not result else f'{result}, {for_freelancer}' if for_freelancer else result
    return result
    
unique_jobs['contract_type'] = unique_jobs.apply(lambda v : extract_job_contract_type(v), axis=1)
freelance_jobs = unique_jobs[unique_jobs['contract_type'].str.contains('freelance', case=False, na=False)]
freelance_jobs.to_excel('./results/freelance_job_offers_19-12-2024.xlsx', index=False)
freelance_jobs

C:\Users\Windows\AppData\Local\Temp\ipykernel_13368\4172168019.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_jobs['contract_type'] = unique_jobs.apply(lambda v : extract_job_contract_type(v), axis=1)


,company_name,company_linkedin_url,job_inf,job_url,job_title,job_mode,company_size,Secteur,job_skills,job_description,key_words,contract_type
2,Black Belt,https://www.linkedin.com/company/black-belt-io...,"Île-de-France, France · il y a 5 jours · 36 ca...",https://www.linkedin.com/jobs/view/4098199516/...,Business Developer freelance,Sur site Temps plein Confirmé,11-50 employés,Recrutement et placement de personnel,"Business, Développement commercial, Gestion de...",À propos de l’offre d’emploi\n\nOpportunité de...,business developer freelance,freelance
157,Bras Droit des Dirigeants,https://www.linkedin.com/company/bras-droit-de...,"Argenteuil, Île-de-France, France · il y a 11 ...",https://www.linkedin.com/jobs/view/4104602535/...,DIRECTEUR COMMERCIAL INDEPENDANT H/F H/F,Sur site Cadre,51-200 employés,Services et conseil aux entreprises,NaN,À propos de l’offre d’emploi\n\nMISSIONS Rejoi...,Directeur Commercial,freelance


In [12]:
def extract_profile_address_from_linkedin_profile_url(self, profile_url):
    try:
        self.driver.get(profile_url)
        time.sleep(random.uniform(2,2.5))
        address_elem =  self.get_element('//*[@id="profile-content"]/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[1]')
        if address_elem["status"]:
            address_elem = address_elem['data']
            address = address_elem.get_attribute('innerText')
            print(address)
            return {"status": True, "data": address }
        else:
            print(address_elem)
            return {"status": False, "data": address_elem['data'] }
    except Exception as e:
        print(f'ERROOOOR: ({profile_url}) {e}')
        return {"status": False, "data": str(e) }

def extract_country(address):
    country = None
    keywords = ['nice', 'lille', 'rouen', 'la réunion', 'chambéry', 'le havre', 'metz', 'dijon', 'clermont-ferrand', 'nantes', 'paris', 'dunkerque',
                'rennes', 'toulouse', 'reims', 'marseille', 'france', 'saint-étienne', 'lyon', 'bordeaux', 'toulon', 'angers', 'montpellier',
                'strasbourg', 'grenoble', 'tours', 'le mans']
    
    list_keys = [[f'{key},', f'{key} ', f' {key},', f' {key} '] for key in keywords]
    new_keywords = [key for list_key in list_keys for key in list_key]
    
    if any(keyword in str(address).lower() for keyword in new_keywords):
        country = 'france'
    
    return country
    
# for dict in dicts[:10]:
#     profile_url = dict.get('profile_url')
#     extract_profile_address_from_linkedin_profile_url(linkedinAutomation, dict.get('profile_url'))

In [14]:
def get_linkedin_page(self, path=None, innerText=None):
    if innerText:
        print('innerText OK')
        a_elem =  self.get_element(path, innerTextLower=str(innerText).lower())
    else:
        a_elem =  self.get_element(path)
        
    if a_elem["status"]:
        a_elem = a_elem['data']
        page_url = a_elem.get_attribute('href')
        # self.driver.get(page_url)
        self.click_elem(a_elem)
        time.sleep(1)
        return {"status": True, "data": page_url }
    else:
        print('error at get_linkedin_page : ', a_elem)
        return {"status": False, "data": a_elem['data']}

In [48]:
import pandas as pd

df = pd.read_excel('./results/job_offers_2_18-12-2024_13h00min.xlsx')
unique_company = df.drop_duplicates(subset=['company_name'])
unique_company.to_excel('./results/unique_company_job_offers_19-12-2024.xlsx', index=False)

In [1]:
from Item import Item
from ItemStorage import ItemStorage
from LinkedinAutomation import LinkedinAutomation
import time, random
import os
from dotenv import load_dotenv
import copy

# Charger les variables d'environnement depuis le fichier .env
load_dotenv('settings.env')

# Accéder aux variables d'environnement
email = os.getenv('LINKEDIN_EMAIL')
pwd = os.getenv('LINKEDIN_PWD')

linkedinAutomation = LinkedinAutomation()
linkedinAutomation.driver.quit()

import undetected_chromedriver as uc
driver = uc.Chrome()
linkedinAutomation.driver = driver

In [4]:
linkedinAutomation.get_linkedin_authentication(email = email, pwd = pwd)

linkedin_authentication --> yes


In [9]:
from_path = "./results/companies_1_02-01-2025.csv"
result_path = "./results/companies_2_02-01-2025.csv"

itemStorage = ItemStorage(file_path= from_path)
dicts_1 = itemStorage.get_list_of_dicts()

if os.path.isfile(result_path):
    itemStorage = ItemStorage(file_path= result_path)
    dicts_2 = itemStorage.get_list_of_dicts()
else :
    dicts_2 = []

def difference_dic(dic1, dic2, column_name=''):
    # Extraire les valeurs de 'Nom_de_l_entreprise' de dic2 dans un set pour une recherche rapide
    entreprises_dic2 = {d[column_name] for d in dic2}
    # Retourner les dictionnaires dans dic1 qui ne sont pas dans dic2 (selon 'Nom_de_l_entreprise')
    return [d for d in dic1 if d[column_name] not in entreprises_dic2]
    
dicts = difference_dic(dicts_1, dicts_2, column_name='company_name')

# dicts.reverse()

# itemStorage = ItemStorage(file_path='./results/editeurs_de_logiciels_11_with_linkedin_about_info_25-11-2024.csv')
# dicts = itemStorage.get_list_of_dicts()

print(len(dicts))

# task = 'extract_contact_information_from_website_url'
# task = 'extract_linkedin_url_from_company_web_site_url'
# task = 'extract_linkedin_url_from_google_based_on_company_name'
task = 'extract_linkedin_about_info'
# task = 'process_linkedin_about_info_column'
# task = 'extract_profiles'
# task = 'extract_founder_and_manager_profiles_based_on_keywords'
# task = 'extract_profile_address_from_linkedin_profile_url'

# number_of_connection = 0
max_number_of_connection = 5
for i, dict in enumerate(dicts):
    items = []
    item = Item(dict)
    
    if task == 'extract_contact_information_from_website_url':
        web_site_url = dict.get('web_site_url')
        about_site_web = dict.get('about site web')
        website_url = web_site_url if str(web_site_url) != 'nan' else str(about_site_web)[2:-2] if str(about_site_web)!='nan' else about_site_web
        results = linkedinAutomation.extract_contact_information_from_website_url(website_url)
        print(results)
        if results['status']:
            item.phones = results['data']['phones']
            item.emails = results['data']['emails']
            item.addresses = results['data']['addresses']
        
    if task == 'extract_linkedin_about_info':
        item = Company(dict)
        result = linkedinAutomation.extract_company_info_from_company_linkedin_profile_url(dict.get('company_linkedin'))
        if result['status']:
            item.linkedin_about_info = result['data']
            print(f'{dict.get('company_name')} ==> {result['data']}')
        else:
            print(f'{dict.get('company_name')} ==> {result['data']}')
            item.linkedin_about_info = None

    if task == 'process_linkedin_about_info_column':
        if str(dict.get('linkedin_about_info')) != 'nan':
            new_dict = { f'about {key}':value for key, value in linkedinAutomation.str_to_json(dict.get('linkedin_about_info')).items()}
            dict.update(new_dict)
        item.init_from_dict(dict)
    #############################################################################################################

    elif task == 'extract_profiles':
        result = linkedinAutomation.extract_profiles_from_company_linkedin_profile_url(dict.get('company_linkedin_url'))
        if result['status']:
            item.profiles = result['data']
            print(f'{dict.get('company_name')} ==> {len(result['data'])}')
        else:
            print(f'{dict.get('company_name')} ==> {result['data']}')
            item.profiles = None

    elif task == 'extract_linkedin_url_from_company_web_site_url':
        result = linkedinAutomation.get_company_linkedin_url_from_company_web_site_url(dict.get('web_site_url'))
        if result['status']:
            item.linkedin = result['data']
            print(f'{dict.get('name')} ==> {result['data']}')
        else:
            print(f'{dict.get('name')} ==> {result['data']}')
            item.linkedin = None

    elif task == 'extract_linkedin_url_from_google_based_on_company_name':
        if i == 0:
            linkedinAutomation.get_google_page()
        result = linkedinAutomation.get_linkedin_url_from_company_name(dict.get('name'))
        if result['status']:
            item.linkedin = result['data']
            print(f'{dict.get('name')} ==> {result['data']}')
        else:
            print(f'{dict.get('name')} ==> {result['data']}')
            item.linkedin = None

    elif task == 'extract_founder_and_manager_profiles_based_on_keywords':
        result = linkedinAutomation.extract_founder_and_manager_profiles_based_on_keywords(dict, for_commercial=True)
        if result['status']:
            print(f'{dict.get('company_name')} ==> {len(result['data'])}')
            for profile in result['data']:
                item.profile_name = profile['profile_name']
                item.profile_description = profile['profile_description']
                item.profile_url = profile['profile_url']
                item.profiles = None
                items.append(copy.deepcopy(item))
        else:
            print(f'{dict.get('name')} ==> {result['data']}')

    elif task == 'extract_profile_address_from_linkedin_profile_url':
        profile_address = extract_profile_address_from_linkedin_profile_url(linkedinAutomation, dict.get('profile_url'))
        if profile_address['status']:
            item.profile_address = profile_address['data']
            item.country = extract_country(item.profile_address)
            if item.country == 'france' and number_of_connection < max_number_of_connection and random.randint(0,20) < 3:
                linkedinAutomation.send_connection_invitation_from_profile_url(linkedinAutomation.driver.current_url)
                number_of_connection += 1
                time.sleep(1)
            print('number_of_connection = ',number_of_connection)
            a = random.randint(0,10)
            print('a == ',a)
            if a <= 6 :
                get_linkedin_page(linkedinAutomation, path='//li[contains(@class, "global-nav__primary-item")]/a', innerText='Réseau')
                linkedinAutomation.scroll_down(button_path_to_show_more_results='//button', button_text_to_show_more_results='afficher plus de résultats', max_scrolling=random.randint(1,3))
                time.sleep(1)
        else:
            item.profile_address = None
    if not items:
        items = item
        
    ItemStorage(file_path=result_path, value=items)
    time.sleep(random.uniform(0.3,1))

765
AXA ==> {'company_size': ['Plus de 10 K employés'], 'site web': ['http://recrutement.axa.fr'], 'page vérifiée': ['1 mars 2023'], 'secteur': ['assurances'], 'taille de l’entreprise': ['10 001 employés et plus', '9 969 membres associés \nmembres linkedin ayant indiqué axa en france comme étant leur lieu de travail actuel sur leur profil.'], 'spécialisations': ['insurance, banque et innovation'], 'vue d’ensemble': ['parmi les leaders de l’assurance, axa accompagne et protège 94 millions de clients dans le monde, qu’ils soient particuliers, professionnels ou entreprises, à chaque étape de leur vie. la protection de nos clients sur le long terme est notre cœur de métier. nous proposons des produits et services d’assurance, de prévoyance, d’assistance, de banque, d’épargne et de protection juridique. notre ambition est de devenir la société préférée de notre secteur d’activité.\naxa en france compte plus de 37 000 collaborateurs. nos équipes sont disponibles, attentionnées et fiables. el

In [26]:
import pandas as pd

df1 = pd.read_excel(from_path)
df2 = pd.read_excel(result_path)

# Sélectionner uniquement la colonne 'company_name' et 'profiles' de df2
df2_reduced = df2[['company_name', 'profiles']]

# Faire la jointure des deux DataFrames sur la colonne 'company_name'
df1 = pd.merge(df1, df2_reduced, on='company_name', how='left')

df1.to_excel("./results/unique_company_4_job_offers_1_500_size_with_profiles_20-12-2024.xlsx",index=False)

In [14]:
itemStorage = ItemStorage(file_path='./results/editeurs_de_logiciels_13_have_between_51_and_1000_employees_with_linkedin_about_info_25-11-2024_19h30min.csv', value=editeurs)
#itemStorage.data = itemStorage.data.loc[itemStorage.data['company_size'].astype(str) != 'nan']
itemStorage.data = itemStorage.data.sort_values(by='index')
itemStorage.file_path = './results/editeurs_de_logiciels_13_have_between_51_and_1000_employees_with_linkedin_about_info_25-11-2024_19h30min.xlsx'
# itemStorage.save_data()

In [45]:
def extract_min_max_size_from_about_company_size_column(df, company_size_column):
    new_df = df # .sort_values(by='index').reset_index(drop=True)
    print('=> unique values ​​in the "company_size_column" :')
    display([str(v).replace('\u202f', '') for v in list(new_df[company_size_column].unique())])
    def get_min_max_size(v, min_or_max):
        try:
            # Nettoyer la valeur
            v_str = str(v).strip().replace('\xa0', '').replace('\u202f', '')
            
            if v_str == 'nan':  # Gérer les valeurs manquantes
                return None
            elif 'Plus de 10 K employés' in v_str or '10001 employés et plus' in v_str:  # Cas spécifique
                return 10000
            elif '-' in v_str:  # Extraire la taille minimale si un intervalle est présent
                if min_or_max == 'min':
                    return int(v_str.split('-')[0].replace('K', '000').strip())
                else:
                    return int(v_str.split('-')[1].replace('K', '000').split('employés')[0].strip())
            else:  # Cas inattendu
                return v_str
        except Exception as e:  # Gérer les erreurs
            return f'errooor : {e}'
    
    new_df['min_size'] = new_df[company_size_column].apply(lambda v: get_min_max_size(v, 'min'))

    print('=> unique values ​​in the "min_size" column :')
    print(list(new_df["min_size"].unique()))
    print()
    
    new_df['max_size'] = new_df[company_size_column].apply(lambda v: get_min_max_size(v, 'max'))

    print('=> unique values ​​in the "max_size" column :')
    print(list(new_df["max_size"].unique()))
    print()
    return new_df

itemStorage = ItemStorage(file_path='./results/unique_company_job_offers_19-12-2024.xlsx')
itemStorage.data = extract_min_max_size_from_about_company_size_column(itemStorage.data, 'company_size')
itemStorage.file_path='./results/unique_company_2_job_offers_19-12-2024.xlsx'
itemStorage.save_data()

=> unique values ​​in the "company_size_column" :


['11-50 employés',
 '2-10 employés',
 '201-500 employés',
 '51-200 employés',
 '10001 employés et plus',
 '501-1000 employés',
 '1001-5000 employés',
 '5001-10000 employés',
 'nan']

=> unique values ​​in the "min_size" column :
[11.0, 2.0, 201.0, 51.0, 10000.0, 501.0, 1001.0, 5001.0, nan]

=> unique values ​​in the "max_size" column :
[50.0, 10.0, 500.0, 200.0, 10000.0, 1000.0, 5000.0, nan]



In [55]:
itemStorage = ItemStorage(file_path='./results/unique_company_2_job_offers_19-12-2024.xlsx')
itemStorage.data['valid_company'] = itemStorage.data['min_size'].apply(lambda v: True if v and v >= 0 and v < 500 else False)
itemStorage.data = itemStorage.data[itemStorage.data['valid_company']]
itemStorage.data['valid_company'] = itemStorage.data['Secteur'].apply(lambda v: True if str(v).lower().strip() not in ['recrutement et placement de personnel', 'services de ressources humaines'] else False)
itemStorage.data = itemStorage.data[itemStorage.data['valid_company']]
itemStorage.file_path='./results/unique_company_3_job_offers_1_500_size_19-12-2024.xlsx'
itemStorage.save_data()

In [61]:
def extract_country(address):
    country = None
    keywords = ['nice', 'lille', 'rouen', 'la réunion', 'chambéry', 'le havre', 'metz', 'dijon', 'clermont-ferrand', 'nantes', 'paris', 'dunkerque',
                'rennes', 'toulouse', 'reims', 'marseille', 'france', 'saint-étienne', 'lyon', 'bordeaux', 'toulon', 'angers', 'montpellier',
                'strasbourg', 'grenoble', 'tours', 'le mans', 'annecy','caen','besançon']
    
    list_keys = [[f'{key},', f'{key} ', f' {key},', f' {key} '] for key in keywords]
    new_keywords = [key for list_key in list_keys for key in list_key]
    
    if any(keyword in str(address).lower() for keyword in new_keywords):
        country = 'france'
    
    return country

itemStorage = ItemStorage(file_path='./results/editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_26-11-2024.csv')
itemStorage.data['country'] = itemStorage.data['profile_address'].apply(extract_country)
itemStorage.file_path = './results/editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_and_country_27-11-2024.xlsx'
# itemStorage.save_data()

In [ ]:
itemStorage.data[itemStorage.data['country']=='france']

In [21]:
# 324 contains (france, paris) + 63  === 387

# 493 with france
# 421 without france
# 420 without france and paris
# 399 without france and paris &&& without france in locations_to_countries

In [13]:
itemStorage = ItemStorage(file_path='./results/editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_26-11-2024.csv')
itemStorage.data = itemStorage.data[itemStorage.data['profile_address'].notna()]
dicts_1 = itemStorage.get_list_of_dicts()
itemStorage = ItemStorage(file_path= "./results/editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_france_country_27-11-2024.csv")
dicts_2 = itemStorage.get_list_of_dicts()

dicts = difference_dic(dicts_1, dicts_2, column_name='profile_url')
editeurs = [copy.deepcopy(EditeurLogiciels(dict)) for dict in dicts]
ItemStorage(file_path='./results/editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_with_country_03-12-2024.csv', value=editeurs)

C:\Users\Windows\OneDrive\Desktop\Projects\Hajy8_editeurs_de_logiciels_Scraping_project\ItemStorage.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data = pd.concat([self.data, new_data], ignore_index=True)


In [69]:
itemStorage = ItemStorage(file_path= "./results/editeurs_de_logiciels_17_have_between_10_and_50_employees_with_linkedin_about_info_02-12-2024.csv")
itemStorage.data['country'] = itemStorage.data['profile_address'].apply(extract_country)
itemStorage.data = itemStorage.data[itemStorage.data['country']=='france']
itemStorage.data['about site web'] = itemStorage.data['about site web'].apply(lambda v: v[2:-2] if str(v)!='nan' else v)
itemStorage.data = itemStorage.data.drop_duplicates(subset=['profile_url'])
itemStorage.file_path= "./results/editeurs_de_logiciels_18_have_between_10_and_50_employees_with_linkedin_about_info_france_country_03-12-2024.csv"
itemStorage.save_data()

In [65]:
list(set(list(itemStorage.data[itemStorage.data['country']!='france']['profile_address'])))

['Autriche ',
 'Philippines ',
 'Dublin, Comté de Dublin, Irlande ',
 'San Diego, Californie, États-Unis ',
 'Région de la baie de San Francisco ',
 'Varsovie et périphérie ',
 'San José, Californie, États-Unis ',
 'New Delhi, Delhi, Inde ',
 'Hammamet, Gouvernorat Nabeul, Tunisie ',
 'Préfecture de Fès, Fès-Meknès, Maroc ',
 'Oviedo, Floride, États-Unis ',
 'Région métropolitaine d’Atlanta ',
 'Israël ',
 'Varsovie, Mazovie, Pologne ',
 'Nashville, Tennessee, États-Unis ',
 'Espagne ',
 'Laval, Québec, Canada ',
 'Abidjan, Côte d’Ivoire ',
 nan,
 'Noida, Uttar Pradesh, Inde ',
 'Saint-Lin, Québec, Canada ',
 'New York, New York, États-Unis ',
 'Mumbai, Maharashtra, Inde ',
 'Tellin, Région wallonne, Belgique ',
 'Miami, Floride, États-Unis ',
 'Milan, Lombardie, Italie ',
 'Canton Neptune, New Jersey, États-Unis ',
 'Andover, Connecticut, États-Unis ',
 'District fédéral, Mexique ',
 'Poznan, Grande Pologne, Pologne ',
 'Liban ',
 'Bengaluru, Karnataka, Inde ',
 'Barcelone, Catalogne,

In [59]:
'Besançon ' in 'Besançon et périphérie'

True

In [71]:
143+97

240

In [37]:
locations_to_countries = {
    'Bruxelles et périphérie ': 'Belgique',
    'Frederiksberg Municipality, Hovedstaden, Danemark ': 'Danemark',
    'Lahore District, Pendjab, Pakistan ': 'Pakistan',
    'Briarcliff Manor, New York, États-Unis': 'États-Unis',
    'Région métropolitaine de Brabant': 'Belgique',
    'Rancho Santa Fe, Californie, États-Unis': 'États-Unis'
    }

locations_to_countries = {key.strip().lower(): value.strip().lower() for key, value in locations_to_countries.items()}

In [45]:
[add for add, country in locations_to_countries.items() if country == 'france']

['dijon et périphérie',
 'clermont-ferrand et périphérie',
 'metz et périphérie',
 'reims et périphérie',
 'bordeaux et périphérie',
 'rennes et périphérie',
 'le mans et périphérie',
 'montpellier et périphérie',
 'grenoble et périphérie',
 'toulon et périphérie',
 'la réunion',
 'nantes et périphérie',
 'rouen et périphérie',
 'lille et périphérie',
 'lyon et périphérie',
 'marseille et périphérie',
 'angers et périphérie',
 'nice et périphérie',
 'le havre et périphérie',
 'chambéry et périphérie',
 'tours et périphérie']

In [39]:
import pandas as pd
df = pd.read_excel('./emelia_results/with_editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_france_country_27-11-2024.xlsx')
df=df[['profile_name','profile_url','profile_description','email_emelia']]

df_1 = pd.read_excel('./EmailListVerify/merged_with_2_editeurs_de_logiciels_06-12-2024_result_1.xlsx')
visited_urls = list(df_1['profile_url'])

filtered_df = df[~df['profile_url'].isin(visited_urls)]
filtered_df = filtered_df.drop_duplicates()
filtered_df.shape

(26, 4)

In [41]:
filtered_df.to_excel('./emelia_results/with_2_editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_france_country_27-12-2024.xlsx', index=False)

In [43]:
import pandas as pd
df1 = pd.read_excel('./emelia_results/with_editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_france_country_27-11-2024.xlsx')
df2 = pd.read_excel('./EmailListVerify/with_2_editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_france_country_27-12-2024_result.xlsx')

# Fusion des deux DataFrames sur les colonnes 'col1', 'col2', et 'col3' 
df_merged = df1.merge(df2, on=['profile_name','profile_url','profile_description','email_emelia'], how='inner')
df_merged.to_excel('./EmailListVerify/merged_with_2_editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_france_country_27-12-2024_result.xlsx', index=False)

In [60]:
import pandas as pd
df1 = pd.read_excel('./EmailListVerify/merged_with_2_editeurs_de_logiciels_06-12-2024_result_1.xlsx')
df2 = pd.read_excel('./EmailListVerify/merged_with_2_editeurs_de_logiciels_14_have_between_51_and_1000_employees_with_linkedin_about_info_france_country_27-12-2024_result.xlsx')

df_concat = pd.concat([df1, df2])
df_concat = df_concat.drop_duplicates(subset=['profile_url'])
df_concat.to_excel('./EmailListVerify/all_files_merged_with_emails_editeurs_de_logiciels_06-12-2024_result.xlsx', index=False)

In [62]:
df_concat

,index,name,tele,address,more_inf_url,email,web_site_url,description,tags,contact,...,profile_description,profile_url,profile_address,country,email_emelia,qualification_emelia,gender_emelia,domain_emelia,ELV Result,ELV Email
0,10,INAGUA solution DAMAaaS,['0181801410'],2 rue du Docteur Vinot 91260 Juvisy France,https://www.archimag.com/societe/inagua-soluti...,contact@damaaas.com,NaN,"INAGUA , la société éditrice de la solution No...","Logiciel de gestion de projet,, Signature éle...",2 rue du Docteur Vinot\n91260\nJuvisy\nFrance\...,...,Président société INAGUA - Editeur du logiciel...,https://www.linkedin.com/in/ronan-bertel-14378...,Paris et périphérie,france,ronan.bertel@damaaas.com,valid,male,damaaas.com,ok_for_all,NaN
1,10,INAGUA solution DAMAaaS,['0181801410'],2 rue du Docteur Vinot 91260 Juvisy France,https://www.archimag.com/societe/inagua-soluti...,contact@damaaas.com,NaN,"INAGUA , la société éditrice de la solution No...","Logiciel de gestion de projet,, Signature éle...",2 rue du Docteur Vinot\n91260\nJuvisy\nFrance\...,...,Founder & CTO @DAMAaaS : La solution universel...,https://www.linkedin.com/in/nicolas-thery-821a...,Lyon et périphérie,france,nicolas.thery@damaaas.com,valid,male,damaaas.com,ok_for_all,NaN
2,10,INAGUA solution DAMAaaS,['0181801410'],2 rue du Docteur Vinot 91260 Juvisy France,https://www.archimag.com/societe/inagua-soluti...,contact@damaaas.com,NaN,"INAGUA , la société éditrice de la solution No...","Logiciel de gestion de projet,, Signature éle...",2 rue du Docteur Vinot\n91260\nJuvisy\nFrance\...,...,Directeur Commercial INAGUA - DAMAaaS platefor...,https://www.linkedin.com/in/fdesnoues?miniProf...,"Viry-Châtillon, Île-de-France, France",france,franck.desnoues@damaaas.com,valid,male,damaaas.com,ok_for_all,NaN
3,10,INAGUA solution DAMAaaS,['0181801410'],2 rue du Docteur Vinot 91260 Juvisy France,https://www.archimag.com/societe/inagua-soluti...,contact@damaaas.com,NaN,"INAGUA , la société éditrice de la solution No...","Logiciel de gestion de projet,, Signature éle...",2 rue du Docteur Vinot\n91260\nJuvisy\nFrance\...,...,Fondatrice Associée DAMAaaS by INAGUA - Dir Mk...,https://www.linkedin.com/in/laurencechapuis?mi...,"Juvisy-sur-Orge, Île-de-France, France",france,laurence.chapuis@damaaas.com,valid,male,damaaas.com,ok_for_all,NaN
4,18,KB Crawl SAS,NaN,NaN,https://www.archimag.com/societe/kb-crawl-sas,NaN,NaN,"KB Crawl est un éditeur de logiciels, spéciali...","Solutions collaboratives, réseau social,, Vei...",NaN,...,Directeur des opérations | Chief Operating Off...,https://www.linkedin.com/in/arnaud-marquant-b8...,Paris et périphérie,france,arnaud.marquant@kbcrawl.com,valid,male,kbcrawl.com,ok,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,342,SER Group,NaN,NaN,https://www.archimag.com/societe/ser-group,NaN,NaN,SER Group est un éditeur leader et un pionnier...,"Logiciel de Ged,, Editeur, distributeur de lo...",NaN,...,Technical partner account manager at SER Group,https://www.linkedin.com/in/thomas-deanaz-5800...,"Lyon, Auvergne-Rhône-Alpes, France",france,thomas.deanaz@sergroup.com,valid,male,sergroup.com,ok,NaN
414,346,Sinequa,NaN,NaN,https://www.archimag.com/societe/sinequa-0,NaN,NaN,Sinequa est un des leaders sur le marché des l...,"Moteur de recherche,, Big data,, Editeur, di...",NaN,...,CEO at Sinequa,https://www.linkedin.com/in/jean-ferre-1802b81...,Paris et périphérie,france,jean.ferre@sinequa.com,valid,female,sinequa.com,ok_for_all,NaN
415,433,Twimm,['+33 1 40 93 99 50'],"8 rue de Saint Cloud, 92150 Suresnes, France, ...",https://www.cloudlist.fr/annuaire/editeurs/log...,contact@twipi-group.com,https://twipi-group.com/solution/twimm-la-gmao...,Twimm est une plateforme full web de GMAO (Ges...,LOGICIELS MÉTIERS,NaN,...,Directeur Général Adjoint,https://www.linkedin.com/in/thierry-kasparian-...,"Suresnes, Île-de-France, France",france,tkasparian@twipi-group.com,valid,male,twipi-group.com,ok,NaN
416,433,Twimm,['+33 1 40 93 99 50'],"8 rue